In [26]:
import numpy as np
import autograd.numpy as anp
import time
from autograd import grad
from scipy.optimize import minimize

def objective_function(x, a, alpha, beta, e):
    try:
        if np.any(np.isnan(x)) or np.any(np.isinf(x)):
            raise ValueError("Input x contains NaN or Inf")
        
        term1 = anp.dot(a, x)
        term2 = alpha * anp.dot(x, x)
        xtx = anp.clip(anp.dot(x, x), 0, 1e10)
        term3 = beta / anp.sqrt(1 + beta * xtx) * anp.dot(e, x)
        result = term1 + term2 + term3
        
        if np.isnan(result) or np.isinf(result):
            raise ValueError("Objective function returned NaN or Inf")
        return result
    except Exception as e:
        print(f"Error in objective_function: {e}")
        raise

def gradient_f(x, a, alpha, beta, e, n):
    try:
        if np.any(np.isnan(x)) or np.any(np.isinf(x)):
            raise ValueError("Input x contains NaN or Inf")
        
        term1 = a
        term2 = 2 * alpha * x
        xtx = anp.clip(anp.dot(x, x), 0, 1e10)
        sqrt_term = anp.sqrt(1 + beta * xtx)
        term3 = (beta / sqrt_term) * e - (beta**2 * anp.dot(e, x) / (sqrt_term**3)) * x
        grad = term1 + term2 + term3
        
        if np.any(np.isnan(grad)) or np.any(np.isinf(grad)):
            raise ValueError("Gradient returned NaN or Inf")
        return grad
    except Exception as e:
        print(f"Error in gradient_f: {e}")
        raise

def project_C(z, n, tol=1e-6):
    try:
        if np.any(np.isnan(z)) or np.any(np.isinf(z)):
            raise ValueError("Input z contains NaN or Inf")
        
        z = np.maximum(np.minimum(z, 1e6), tol)
        
        def obj(x):
            return 0.5 * anp.sum((x - z)**2)
        def constraint(x):
            return anp.sum(anp.log(x))
        constraints = {'type': 'ineq', 'fun': constraint}
        bounds = [(tol, None)] * n
        
        x_init = np.maximum(z, tol)
        if anp.sum(anp.log(x_init)) < 0 or np.any(x_init <= 0):
            geo_mean = np.prod(z) ** (1/n)
            x_init = np.ones(n) * max(tol, geo_mean * np.exp(0.1))
        
        result = minimize(obj, x0=x_init, constraints=constraints, bounds=bounds, method='SLSQP',
                         options={'ftol': 1e-8, 'maxiter': 2000, 'disp': False})
        if not result.success:
            print(f"Projection failed: {result.message}, z={z[:5]}...")
            x_init = np.ones(n) * max(tol, np.prod(z) ** (1/n) * np.exp(0.2))
            result = minimize(obj, x0=x_init, constraints=constraints, bounds=bounds, method='SLSQP',
                            options={'ftol': 1e-8, 'maxiter': 2000, 'disp': False})
            if not result.success:
                raise RuntimeError(f"Projection fallback failed: {result.message}")
        
        return result.x
    except Exception as e:
        print(f"Error in project_C: {e}")
        raise

def gda_adaptive_algorithm(n, max_iter=10000, seed=0):
    np.random.seed(seed)
    beta = 0.741271
    alpha = 3 * (beta ** (3/2)) * np.sqrt(n) + 1
    L = 4 * (beta ** (3/2)) * np.sqrt(n) + 3 * alpha

    a = np.random.uniform(1, 10, n)
    e = np.arange(1, n + 1, dtype=float)
    
    x_k = np.random.uniform(0.5, 1.5, n)
    x_k = project_C(x_k, n)
    
    lambda_k = 1/L  # Giảm bước ban đầu
    sigma = 0.1
    kappa = 0.9
    max_armijo_iter = 20

    start_time = time.time()

    for k in range(max_iter):
        grad_k = gradient_f(x_k, a, alpha, beta, e, n)
        grad_norm = np.linalg.norm(grad_k)
        
        if grad_norm < 1e-3:  # Thêm điều kiện dừng dựa trên gradient norm
            print(f"Stopped due to small gradient norm: {grad_norm:.6f}")
            break
            
        f_current = objective_function(x_k, a, alpha, beta, e)
        lambda_current = lambda_k
        armijo_iter = 0
        
        while armijo_iter < max_armijo_iter:
            step = np.minimum(lambda_current * np.linalg.norm(grad_k), 1.0) * grad_k / np.linalg.norm(grad_k) if np.linalg.norm(grad_k) > 0 else grad_k
            x_candidate = x_k - step
            x_next = project_C(x_candidate, n)
            f_next = objective_function(x_next, a, alpha, beta, e)
            armijo_rhs = f_current - sigma * np.dot(grad_k, x_k - x_next)
            
            if f_next <= armijo_rhs:
                break
            else:
                lambda_current *= kappa
                armijo_iter += 1
        
        if armijo_iter >= max_armijo_iter:
            step = np.minimum(lambda_current * np.linalg.norm(grad_k), 1.0) * grad_k / np.linalg.norm(grad_k) if np.linalg.norm(grad_k) > 0 else grad_k
            x_next = project_C(x_k - step, n)

        if lambda_current < 1e-10:
            print(f"Step size too small at iteration {k}")
            break

        if np.linalg.norm(x_next - x_k) < 1e-8:
            break

        x_k = x_next
        lambda_k = lambda_current

    elapsed_time = time.time() - start_time
    f_final = objective_function(x_k, a, alpha, beta, e)
    
    grad_final = gradient_f(x_k, a, alpha, beta, e, n)
    grad_norm = np.linalg.norm(grad_final)
    kkt_violation = np.any(grad_final[x_k <= 1e-6] < -1e-6)
    print(f"KKT violation = {kkt_violation}, grad norm = {grad_norm:.6f}, product x_i = {np.prod(x_k):.6f}")
    
    return x_k, f_final, k + 1, elapsed_time, a, e

def gd_algorithm(n, max_iter=10000, seed=0, a=None, e=None):
    np.random.seed(seed)
    beta = 0.741271
    alpha = 3 * (beta ** (3/2)) * np.sqrt(n) + 1
    L = 4 * (beta ** (3/2)) * np.sqrt(n) + 3 * alpha
    lambda_k = 1 / L

    if a is None:
        a = np.random.uniform(1, 10, n)
    if e is None:
        e = np.arange(1, n + 1, dtype=float)
        
    x_k = np.random.uniform(0.5, 1.5, n)
    x_k = project_C(x_k, n)

    start_time = time.time()
    for k in range(max_iter):
        grad = gradient_f(x_k, a, alpha, beta, e, n)
        grad_norm = np.linalg.norm(grad)
        
        if grad_norm < 1e-3:
            print(f"Stopped due to small gradient norm: {grad_norm:.6f}")
            break
            
        step = np.minimum(lambda_k * np.linalg.norm(grad), 1.0) * grad / np.linalg.norm(grad) if np.linalg.norm(grad) > 0 else grad
        x_next = project_C(x_k - step, n)
        
        if np.linalg.norm(x_next - x_k) < 1e-8:
            break
        x_k = x_next
        
    elapsed_time = time.time() - start_time
    f_final = objective_function(x_k, a, alpha, beta, e)
    
    grad_final = gradient_f(x_k, a, alpha, beta, e, n)
    grad_norm = np.linalg.norm(grad_final)
    kkt_violation = np.any(grad_final[x_k <= 1e-6] < -1e-6)
    print(f"KKT violation = {kkt_violation}, grad norm = {grad_norm:.6f}, product x_i = {np.prod(x_k):.6f}")
    
    return x_k, f_final, k + 1, elapsed_time

def run_comparison(n):
    print("So sánh thuật toán GDA vs GD")
    print("="*60)

    print(f"\nChạy cho n = {n}:")
    
    x_opt_gda, f_opt_gda, iters_gda, time_gda, a, e = gda_adaptive_algorithm(n)
    print(f"GDA: f(x*) = {f_opt_gda:.6f}, #Iter = {iters_gda:4d}, Time = {time_gda:.4f}s")
    
    x_opt_gd, f_opt_gd, iters_gd, time_gd = gd_algorithm(n, a=a, e=e)
    print(f"GD : f(x*) = {f_opt_gd:.6f}, #Iter = {iters_gd:4d}, Time = {time_gd:.4f}s")
    
    print("\n" + "="*80)
    print(f"KẾT QUẢ CHO n = {n}")
    print("="*80)
    print("| n   | GDA: f(x*)  | #Iter | Time (s)  | GD: f(x*)   | #Iter | Time (s)  |")
    print("|-----|-------------|-------|-----------|-------------|-------|-----------|")
    print(f"| {n:<3} | {f_opt_gda:>11.6f} | {iters_gda:>5} | {time_gda:>9.4f} | {f_opt_gd:>11.6f} | {iters_gd:>5} | {time_gd:>9.4f} |")

if __name__ == "__main__":
    n = 2
    run_comparison(n)

So sánh thuật toán GDA vs GD

Chạy cho n = 2:
KKT violation = False, grad norm = 20.388686, product x_i = 1.000000
GDA: f(x*) = 22.157584, #Iter =   31, Time = 0.2679s
KKT violation = False, grad norm = 20.388686, product x_i = 1.000000
GD : f(x*) = 22.157584, #Iter =   18, Time = 0.0988s

KẾT QUẢ CHO n = 2
| n   | GDA: f(x*)  | #Iter | Time (s)  | GD: f(x*)   | #Iter | Time (s)  |
|-----|-------------|-------|-----------|-------------|-------|-----------|
| 2   |   22.157584 |    31 |    0.2679 |   22.157584 |    18 |    0.0988 |
